In [ ]:
OPENAI_API_KEY = ""

from openai import OpenAI 
import os
import base64

MODEL="gpt-4o-mini"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OPENAI_API_KEY))

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def extract_text(base64_image):
    response = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can extract text from an image and return in a latex format"},
            {"role": "user", "content": [
                {"type": "text", "text": "Can you extract all the text from this image?"},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"}
                }
            ]}
        ],
        temperature=0.0,
    )

    return response.choices[0].message.content

def save_to_markdown(text, path):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    
    print(f"saved to {path}")

In [2]:
from natsort import natsorted
from tqdm.auto import tqdm
import os

FOLDER_PATH = "../data/images"
OUTPUT_PATH = "../data/latex_format/"

for _file in tqdm(natsorted(os.listdir(FOLDER_PATH))):
    text = ''
    for _image in natsorted(os.listdir(os.path.join(FOLDER_PATH, _file))):
        base64_image = encode_image(os.path.join(FOLDER_PATH, _file, _image))
        text += extract_text(base64_image)

    save_to_markdown(text, OUTPUT_PATH + _file + '.md')

e:\pi_school\eva-work-notes\text-extraction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  5%|▌         | 1/20 [01:35<30:21, 95.89s/it]

saved to ../data/latex_format/000cd93a-cecf-46f7-9ea9-a316b2000641.md


 10%|█         | 2/20 [09:42<1:37:47, 325.98s/it]

saved to ../data/latex_format/1a0acabc-5140-425b-9eb8-f90e1721a6c3.md


 15%|█▌        | 3/20 [10:48<58:41, 207.15s/it]  

...

saved to ../data/latex_format/ff36bbf0-e71c-493e-834d-94906b03d7cc.md


 95%|█████████▌| 19/20 [1:54:46<07:16, 436.11s/it]

saved to ../data/latex_format/mar-18-eos-magazine.md


100%|██████████| 20/20 [2:00:27<00:00, 361.37s/it]

saved to ../data/latex_format/s44288-024-00042-0.md
